## Introduction to Specviz

#### Learning Goals

- Learn to load and display a spectrum in Specviz
- Become familiar with the tools attached to the spectrum viewer
- Become familiar with the data analysis plugins
- Learn how to select a region of interest/subset of the spectrum
- Learn how to overlay spectral lines on the displayed spectrum
- Learn how to extract spectra from the Specviz app for further analysis in the notebook


In [ ]:
# This ensures that our notebook is using the full width of the browser

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Running the app

To initialize the Specviz app, we import the `Specviz` class, initialize an instance of it, and invoke the `app` property to display the app. We also import a few more things that will be useful later.

In [ ]:
from jdaviz import SpecViz

from astropy.utils.data import download_file
import astropy.units as u

specviz = SpecViz()
specviz.app

### Loading data into the app

Loading data is as simple as calling the `load_spectrum` method of the `Specviz` class. Here, we download an SDSS spectrum and load it into the app, making use of the optional `data_label` argument to give the data a sensible label, and using the `format` argument to tell the `specutils` data loader on the backend that this is an SDSS file. Note that `load_spectrum` can also be given `specutils.Spectrum1D` objects as input. 

In [ ]:
fn = download_file('https://data.sdss.org/sas/dr14/sdss/spectro/redux/26/spectra/0751/spec-0751-52251-0160.fits', cache=True)
specviz.load_spectrum(fn, data_label="Demo SDSS", format="SDSS-III/IV spec")

### Selecting a region of interest

First, note that before selecting any subsets, the subset selection button in the app-level toolbar reads "No selection (create new)". This indicates that using the x-range selection tool will create a new subset, rather than editing an existing subset. To the right of that button you should see a small circle - clicking that circle shows that there are options for what the x-range selection will do. Since we're creating a new subset, the default ("replace") is fine.

Follow these steps to select a new region:
1. Click the "hammer and screwdriver" icon in the top left of the viewer. 
2. Use any of the pan/zoom tools (the leftmost three icons in the expanded tool tray) to zoom into the feature of interest. We'll investigate the broad emission feature near 6900 Angstrom. Zooming works by scrolling with the mouse wheel (see "Sidebar: zooming from the notebook" below if you can't scroll to zoom)
3. Click the "x-range" selection tool in the expanded tool tray.
4. Click and drag on the viewer to select a region around the feature.
5. Click the "x-range" button in the expanded tool tray to deselect the tool.

You should now see the label "Subset 1" with a red triangle in the app-level toolbar where it previously said "No selection (create new)". The red triangle indicates that the region of the spectrum you selected in the viewer should be colored red to indicate it is in this subset.

#### Sidebar: zooming from the notebook

If you don't have the capability to zoom via scrolling with a zoom tool selected, you can set the limits of the x and y axes manually from the notebook by calling e.g.:

specviz.x_limits(6200, 7500)

specviz.y_limits('auto', 120)

### Adding line lists via the notebook

Line lists are expected to be input as an astropy `QTable`, with at minimum "linename" and "rest" fields. You can optionally specify a "color" column if you want each line to be a different color. 

The first step is to load the line list into the specviz instance using `specviz.load_line_list(lines)`. A line list as described as above is required, and there is an optional `replace` parameter that, if set to True, replaces any existing lines with those input. The default behavior is to add newly loaded lines to the existing list.

Note that the line list functionality is all actually happening under the hood in the spectrum viewer (and thus could also be used in e.g. Cubeviz); the Specviz methods are convenience wrappers. 

In [ ]:
from astropy.table import QTable
import astropy.units as u

lt = QTable()
lt['linename'] = ['Test1', 'Test2']
lt['rest'] = [6.8e-7, 4.4e-7]*u.m
lt['redshift'] = u.Quantity(0.046)
lt['listname'] = "Test"

specviz.load_line_list(lt)

Displaying the lines on the plot is as simple as calling the `plot_spectral_lines` function:

In [ ]:
specviz.plot_spectral_lines()

And one can erase all spectral lines from the plot as below:

In [ ]:
specviz.erase_spectral_lines()

### Using the Line Lists plugin

We can also add and remove lines via the Line Lists plugin, including a set of predefined line lists that can be loaded and plotted by the user. These are viewable in the plugin under the "Available Line Lists" dropdown. Once a list is added, you can open the list menu and choose to display and hide either individual lines from the list or all the lines in the list. Note that showing more than a dozen or so lines at once can start impacting the performance of the redshift slider.

### Retrieving data

The data from the app can be retrieved into the notebook using the `get_spectra` method of the `Specviz` class. Retrieve the data using that method below. Note that the returned objects are `specutils.Spectrum1D` instances, which you learned about in the previous session.

## Exercises

#### Basic

1. Delete Subset 1. Use the subset selection tool to create select the broad emission feature again, but this time remove the narrow emission features evident on  top of the broad feature from the subset. See how the calculated centroid value changes in the Line Analysis plugin. 
2. Use the Unit Conversion plugin to change the units of the displayed spectrum. Then use the data selection menu to switch back to the original spectrum.

#### Advanced

1. Create a subset that includes some continuum on either side of the broad emission feature, then use the Model Fitting plugin to fit a constant model to that subset. Using `specviz.get_spectra()`, retrieve the data from the app and use the continuum fit to normalize the spectrum. Load the normalized spectrum back into the app.

    **Hint**: The "Model ID" of any component model you initialize in the Model Fitting plugin needs to be used in the `Model Equation Editor` section of the plugin before pressing the `Fit` button.